In [1]:
# change workdir to be root directory of covariance_resolution software
workdir = '/home/patrick/Master_Lenovo/Academic/ORNL Research Position (August 2016 - June 2017)/linear_Gaussian_error_propagation/covariance_resolution_method'
%cd {workdir}

/home/patrick/Master_Lenovo/Academic/ORNL Research Position (August 2016 - June 2017)/linear_Gaussian_error_propagation/covariance_resolution_method


In [2]:
# Import necessary python modules
import numpy as np
from matplotlib import pyplot as plt

import compute_covariance_matrix as cov_mat
import ARCS_error_analysis as arcs
import compute_vQ_to_HKL_basis_change_matrix as HKL_basis
import plot_covariance_ellipse as plot_cov
import avg_cov_mat_across_angle_sweep as avg_cov


# define physical constants:

# m = mass of neutron (kg)
m = 1.674929*(10**-27)

# hbar = reduced Planck's constant (J s)
hbar = 1.0545718*(10**-34)

First, we need to define the required instrumental parameters for our experiment:

In [3]:
# set instrument (currently, only arcs supported)
instrument = "arcs"

# a bit of hacking:
# define distances (eventually, this will be done automatically)
L12 = 18.50 - 11.83  # distance from beam monitor 1 to 2
Lms = 13.60  # distance from moderator to sample
Lsp = 3.45  # distance from sample to detector pixel -- this will also need to be changed eventually to take a particular values


# incident beam energy
Ei_meV = 100.0  # meV

# set u, v sample orientation vectors
#u = [1, 0, 2]
#v = [1, 0, 0]
u = [-1, 1, -1]
v = [2, 1, -1]

Next, we need to define our sample properties

In [4]:
# lattice parameters
a = 5.4907
b = 5.4907
c = 5.4907

# lattice vectors
a1 = np.array([a, 0, 0])
a2 = np.array([0, b, 0])
a3 = np.array([0, 0, c])

# create a list of lattice vectors
lattice = [a1, a2, a3]

# sample rotation angle
#angle = 44.3427
#angle = 80.0
angle = -60.0 #44.3427

Now, we will take a specified point in HKLE space as our input, then compute the corresponding instrument coordinates

In [5]:
# specify HKLE point:
#HK = 1.5
#H = np.sqrt(HK**2 / 2.0)
#K = H
#L = 0.0
#scale = -5. -1./3
#H = 1.0*scale
#K = .5*scale
#L = -.5*scale
H = -2.
K = -2.
L = 0.
#H = -H #1.0*scale
#K = -H #.5 * scale
#L = H#-.5 * scale
#E = 13.7 # meV
#E = 3.50
#E=35.0
E = 0.0

# specify this as a linear combination for plotting purposes:
#x = np.array([H, K, L, 0.0])
#x = np.array([-H, H, -H, 0.0])
#x = np.array([1.2, 3.4, 0.4, 0.0])
#x = np.array([1.0, 0.0, 0.0, 0.0])

# now, convert to real units (inverse angstroms)
h = H * 2*np.pi / a
k = K * 2*np.pi / b
l = L * 2*np.pi / c
#x = np.array([-1, 1, -1, 0.0])
x = np.array([0.0, 0., 1., 0.])

# make array of hklE values
HKLE = np.array([h, k, l, E])

# compute instrument coordinates
instr_coords = avg_cov.HKLE_to_instrumentCoords(lattice, u, v, angle, HKLE, Ei_meV, L12, Lms, Lsp)

print (instr_coords)



[0.46384797354740143, 0.49195221134180817, 0.001524943631755321, 0.003898094291068699]


Extra Information:  This section will be refined later to be more user-friendly.  Right now, it is just an information dump of particular details for which point in instrument space / Q,E space / HKLE space is going to be analyzed

In [6]:
theta = instr_coords[0]
phi = instr_coords[1]
t = instr_coords[3]

print(t)

0.00389809429107


We now need to actually compute the covariance matrix

In [7]:
# get Jacobian matrix
#J = arcs.setup_jacobian(vi, vf, Ei, Ef, theta, phi, L12, Lms, Lsp, tof, t12)
#J = arcs.setup_jacobian(vi, vf, Ei, Ef, theta, phi, t, t12)

# get instrumental parameters matrix
#M = cov_mat.setup_params_matrix(var_t12, var_theta, var_phi)
JM = arcs.get_jacobian_and_params_matrices_from_event_data(t, theta, phi, Ei_meV)
J = JM[0]
M = JM[1]

# get vQE covariance matrix
Sigma_vQE = cov_mat.get_sigma_vQE(J, M)
SigmaInv_vQE = cov_mat.get_sigma_inv_vQE(J, M)

# get HKLE covariance matrix
Sigma_HKLE = cov_mat.get_sigma_HKLE(Sigma_vQE, lattice, u, v, angle)

# get inverse HKLE covariance matrix (for plotting)
SigmaInv_HKLE = cov_mat.get_sigma_inv_HKLE(Sigma_HKLE)

Reduced_Sigma = cov_mat.get_2D_sliced_covariance(Sigma_HKLE, x)
Reduced_SigmaInv = cov_mat.get_2D_sigma_inv(Reduced_Sigma)

t12 = 0.00152494363176
E_partial_t12 = -9.22766194112e-17



In [8]:
print("Sigma_vQE = \n")
print(Sigma_vQE)

print("\nSigma_HKLE = \n")
print(Sigma_HKLE)

print("\nReduced_Sigma = \n")
print(Reduced_Sigma)

Sigma_vQE = 

[[  5.89532197e-04   2.31139959e-04  -1.35779604e-04  -4.26989692e-03]
 [  2.31139959e-04   2.82086693e-04  -7.27640705e-05  -2.28823086e-03]
 [ -1.35779604e-04  -7.27640705e-05   6.12226038e-04  -2.60835358e-02]
 [ -4.26989692e-03  -2.28823086e-03  -2.60835358e-02   2.36270197e+00]]

Sigma_HKLE = 

[[  6.35129345e-04  -8.01843710e-05  -2.11103952e-04   1.97498723e-02]
 [ -8.01843710e-05   5.43642923e-04   1.76436327e-04   1.17576978e-02]
 [ -2.11103952e-04   1.76436327e-04   3.05072660e-04  -3.02853367e-03]
 [  2.58623865e-02   1.53966628e-02  -3.96585391e-03   2.36270197e+00]]

Reduced_Sigma = 

[[  3.05072660e-04  -3.02853367e-03]
 [ -3.96585391e-03   2.36270197e+00]]


In [9]:
# select two variables (from H,K,L,E) to plot: (0,1,2,3 = H,K,L,E)
x1 = 0
x1_title = ""
x2 = 3
x2_title = ""

if x1 == 0:
    x1_title = "H-direction (A^-1)"
elif x1 == 1:
    x1_title = "K-direction (A^-1)"
elif x1 == 2:
    x1_title = "L-direction (A^-1)"
elif x1 == 3:
    x1_title = "E (meV)"

if x2 == 0:
    x2_title = "H-direction (A^-1)"
elif x2 == 1:
    x2_title = "K-direction (A^-1)"
elif x2 == 2:
    x2_title = "L-direction (A^-1)"
elif x2 == 3:
    x2_title = "E (meV)"
    
# extract submatrix:
A = plot_cov.get_2D_subcovariance(SigmaInv_vQE, x1, x2)

# specify degrees of freedom and significance level
alpha = 0.5
k = 2
# compute critical chi-squared value
chi2 = plot_cov.get_critical_chi_squared(k, alpha)

# plot covariance matrix:
#plot_cov.plot_quadform(A, x1, x2, chi2, x1_title, x2_title, 0)
plot_cov.plot_quadform(Reduced_Sigma, x1, x2, chi2, "x direction (A^-1)", x2_title, 0)
#plot_cov.plot_quadform_method2(A, x1, x2, chi2, x1_title, x2_title)